1. Lese die `EXP_UNIQUE_ID` aus und finde die korrespondierende Hyperparamerterkonfiguration in `05_done_workloads.csv`
2. Nehme nur diejenigen Reihen auf, welche eine `EXP_BATCH_SIZE` von **5** haben (es gilt: `EXP_BATCH_SIZE` kann nur folgende Werte einnehmen `[1,5,10]`)
3. Prüfe ob alle selektierten Dataframes nur Werte ≠ `NaN` beinhalten, wenn nicht, interpoliere die zu füllenden Einträge
   auf Grundlage unserer gewähten Interpolationsstrategie **(TODO)**
4. Führe mit allen Datenvektoren PCA durch um eine Dimensionsreduktion zu erzielen (beschleunigt Clustering) 
   $\rightarrow$ Dimensionen sollten danach für alle Datenvektoren gleich sein! 
5. Führe ein k-means Clustering durch, experimentiere dabei mit verschieden großen k um "bestes" Clustering zu finden
6. Führe ein Clustering durch, welches **OHNE** vorherige Initialisierung der Clusteranzahl und ähnliches auskommt
   $\rightarrow$ (**TODO:** Recherche welche Strategie)

In [1]:
import pandas as pd
import os
from typing import Dict, List

base:str = "kp_test/"
strategys:List[str] = sorted([ (strat + "/") for strat in os.listdir(base) if strat[0].isupper()], key=str.lower)
datasets:List[str] = sorted([ (dset + "/") for dset in os.listdir(base + strategys[0])], key=str.lower)
metrices:List[str] = sorted([met for met in os.listdir(base+strategys[0]+datasets[0])], key=str.lower)

HYPERPARAMETERS:pd.DataFrame = pd.read_csv(base + "05_done_workload.csv")
BATCHES:Dict[int, List[List[float]]] = {1:list(), 5:list(), 10:list()}

for strategy in strategys:
    for dataset in datasets:
        # metrices[0] should contain the accuracy metric regarding the sorting key
        df = pd.read_csv(base + strategy + dataset + metrices[0])
        for index, row in df.iterrows():
            # last entry in every row contains EXP_UNIQUE_ID
            EXP_UNIQUE_ID:int = int(row[-1])
            # get the batch size out of the row from hyperparamter configuration 
            batch_size:int  = int(HYPERPARAMETERS[HYPERPARAMETERS["EXP_UNIQUE_ID"] == EXP_UNIQUE_ID]["EXP_BATCH_SIZE"].iloc[0])
            # store value-vector in the dictionary
            BATCHES[batch_size].append(row.dropna().to_list()[:-1])
        # check if all entrie vectors have the same length:
        for key, value in BATCHES.items():
            try:
                assert(all(len(vec) == len(vec[0])) for vec in value)
            except:
                raise Exception("Vectors have not the same length!")
        break
    break

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,EXP_UNIQUE_ID
0,0.950000,0.666667,0.666667,0.983333,1.000000,0.95,0.950000,1.000000,0.966667,0.950000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,50
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61
2,0.950000,0.783333,0.800000,0.800000,0.800000,1.00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,66
3,0.783333,0.833333,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83
4,0.866667,0.583333,0.866667,0.850000,0.866667,0.95,0.966667,0.966667,0.966667,0.983333,...,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,149
